In [1]:
import os
import pandas as pd

TUXML_CSV_FILENAMES = ['config_bdd30000-40000.csv', 'config_bdd40000-60000.csv', 'config_bdd60000-90000.csv', 'config_bdd90000-100000.csv']
# ['config_bdd30000-40000.csv', 'config_bdd90000-100000.csv']

#pd_chunks = []
#for csv_filename in TUXML_CSV_FILENAMES:
    # sanity check CSV
    #with open(csv_filename, "r") as file:
    #    k = file.readline()
    #    t = k.split(",")
    #    s = set(t)
    #    assert(len(t) == len(s)) # unique number of options/features/column names

    # parsing for real with pandas 
    #c_pd = pd.read_csv(open(csv_filename, "r"))
    #pd_chunks.append(c_pd)
    #print("DONE for", csv_filename)  
pd_csvs = [pd.read_csv(f) for f in TUXML_CSV_FILENAMES]
combined_pd = pd.concat(pd_csvs)

/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1150,6015,6026,6717,7350,7676,7726,10442) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1150,6015,6026,6717,7350,7676,7726,8507,10442) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1150,2722,6015,6026,6717,7350,7676,7726,9949) have mixed types. Specify dtype option on import or set low_memory=False.
  if (yield from self.run_code(code, result)):
/usr/local/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3185: DtypeWarning: Columns (1150,6015,6026,6717,7350,7676,7726) have mixed types. Specify dtype option on import or set low_me

In [2]:
combined_pd.to_pickle('config_bdd30-100.pkl')

In [3]:
# assert (len(combined_pd) == 20001)
len(combined_pd)

69459

In [4]:
len(pd_csvs[0]), len(pd_csvs[1]), len(pd_csvs[2]), len(pd_csvs[3])

(9939, 19683, 29837, 10000)

In [6]:
basic_head = ["cid", "time", "date"] # "compile"
size_methods = ["vmlinux", "GZIP-bzImage", "GZIP-vmlinux", "GZIP", "BZIP2-bzImage", 
              "BZIP2-vmlinux", "BZIP2", "LZMA-bzImage", "LZMA-vmlinux", "LZMA", "XZ-bzImage", "XZ-vmlinux", "XZ", 
              "LZO-bzImage", "LZO-vmlinux", "LZO", "LZ4-bzImage", "LZ4-vmlinux", "LZ4"]

In [10]:
### basic stats about options and unique values 
## could be extended/improved 

tri_state_values = ['y', 'n', 'm']

ftuniques = []
freq_ymn_features = []
non_tristate_options = []

for col in combined_pd:
    ft = combined_pd[col]    
    # eg always "y"
    if len(ft.unique()) == 1:
        ftuniques.append(col)
    # only tri-state values (y, n, m) (possible TODO: handle numerical/string options)    
    elif all(x in tri_state_values for x in ft.unique()):     #len(ft.unique()) == 3: 
        freq = ft.value_counts(normalize=True)
        freqy = 0
        freqn = 0
        freqm = 0
        if ('y' in freq.index):
            freqy = freq['y']
        if ('n' in freq.index):
            freqn = freq['n']
        if ('m' in freq.index):
            freqm = freq['m']
        freq_ymn_features.append((col, freqy, freqm, freqn))
    else:
        if not (col in size_methods): 
            non_tristate_options.append(col)
        

### TODO: we want to keep all quantitative values!
# non_tristate_options.remove('LZO') # ('vmlinux')

# we want to keep measurements (that are not tristate ;)) 
# non_tristate_options = list(set(non_tristate_options) - set(size_methods))

#### print options with unique values
# options with only one value eg always "y"
#i = 0
#for ft in ftuniques:
#    print(ft + " (" + str(i) + ")")
#    i = i + 1

print("Original size (#configs/#options) of the dataset " + str(combined_pd.shape))
print ("Number of options with only one value (eg always y): " + str(pd.DataFrame(ftuniques).shape))

# maybe we can drop options with only one unique value (no interest for machine learning)
# TODO: maybe we can rely on more traditional feature reduction techniques
# TODO: need to think about *when* to apply the removal 

#rawtuxdata.drop(columns=ftuniques,inplace=True) 

## non_tristate_options include basic stuff like date, time, cid but also string/numerical options
print ("Non tri-state value options (eg string or integer or hybrid values): " 
       + str(pd.DataFrame(non_tristate_options).shape) + " ") 
#      + str(pd.DataFrame(non_tristate_options)))


print ("Predictor variables: " + str(combined_pd.drop(columns=non_tristate_options).columns.size))
# frequency of y, m, and n values 
#plt.figure()
#pd.DataFrame(freq_ymn_features, columns=["feature", "freqy", "freqm", "freqn"]).plot(kind='hist', alpha=0.8) #plot()
#plt.show()


combined_pd.info(memory_usage='deep')

Original size (#configs/#options) of the dataset (69459, 12798)
Number of options with only one value (eg always y): (2885, 1)
Non tri-state value options (eg string or integer or hybrid values): (174, 1) 
Predictor variables: 12624
<class 'pandas.core.frame.DataFrame'>
Int64Index: 69459 entries, 0 to 9999
Columns: 12798 entries, cid to NETWORK_FILESYSTEMS
dtypes: float64(30), int64(234), object(12534)
memory usage: 53.6 GB


In [12]:
from sklearn.preprocessing import *
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LassoCV
from sklearn.pipeline import Pipeline
from sklearn import linear_model
from sklearn import svm
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn import tree
import graphviz 
import numpy as np


LEARN_COMPILATION_SUCCESS = True # costly in time and space 
compilation_status_column_name = 'compile_success'

def encode_data_compilation(rawtuxdata):
    lae = LabelEncoder()
    # we save quantitative values we want (here vmlinux, TODO: generalize)
    # the key idea is that the labelling encoder should not be applied to this kind of values (only to predictor variables!)
    # vml = rawtuxdata['LZO'] # rawtuxdata['vmlinux'] 
    o_sizes = rawtuxdata[size_methods]

    # we may remove non tri state options, but TODO there are perhaps some interesting options (numerical or string) here
    #tuxdata = rawtuxdata.drop(columns=non_tristate_options).drop(columns=['vmlinux']).apply(le.fit_transform)
    tuxdata_for_compilation = rawtuxdata.drop(columns=non_tristate_options).drop(columns=size_methods).apply(lae.fit_transform)

    #tuxdata['vmlinux'] = vml 
    tuxdata_for_compilation[size_methods] = o_sizes
    # we can ue vmlinux since it has been restored thanks to previous line
    tuxdata_for_compilation[compilation_status_column_name] = tuxdata_for_compilation['vmlinux'] != -1
    return tuxdata_for_compilation

def learn_compilation_success(tuxdata_for_compilation):
    TESTING_SIZE=0.1 
    X_train, X_test, y_train, y_test = train_test_split(tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name), tuxdata_for_compilation[compilation_status_column_name], test_size=TESTING_SIZE, random_state=0)  
    clf = tree.DecisionTreeClassifier() #GradientBoostingClassifier(n_estimators=100) #RandomForestRegressor(n_estimators=100) #   #GradientBoostingRegressor(n_estimators=100)  
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    importances = clf.feature_importances_
    indices = np.argsort(importances)[::-1]    

    TOP_FT_IMPORTANCE=20
    print("Feature ranking: " + "top (" + str(TOP_FT_IMPORTANCE) + ")")
    for f in range(TOP_FT_IMPORTANCE): # len(indices)
        print("%d. feature %s %d (%f)" % (f + 1, tuxdata_for_compilation.columns[indices[f]], indices[f], importances[indices[f]]))
   
    
    dot_data = tree.export_graphviz(clf, out_file=None, 
                         feature_names=tuxdata_for_compilation.drop(columns=size_methods).drop(columns=compilation_status_column_name).columns,  
                         filled=True, rounded=True,
                         special_characters=True)  
    graph = graphviz.Source(dot_data)  
    graph.render("TUXML_compilation_failures")
    
    acc = accuracy_score (y_test, y_pred)
    prec = precision_score (y_test, y_pred)
    reca = recall_score (y_test, y_pred)
    f1 = f1_score (y_test, y_pred)
    print("Accuracy score: %.2f" % (acc))
    print("Precision score: %.2f" % (prec))
    print("Recall score: %.2f" % (reca))
    print("F1 score: %.2f" % (f1))

if (LEARN_COMPILATION_SUCCESS):
    tuxdata_for_compilation = encode_data_compilation(combined_pd)
    tuxdata_for_compilation [compilation_status_column_name].describe() # TODO?
    learn_compilation_success(tuxdata_for_compilation)
    del tuxdata_for_compilation

Feature ranking: top (20)
1. feature AIC79XX_BUILD_FIRMWARE 2536 (0.444362)
2. feature AIC7XXX_BUILD_FIRMWARE 1789 (0.305322)
3. feature WANXL_BUILD_FIRMWARE 10508 (0.126132)
4. feature DRM_VBOXVIDEO 7999 (0.075337)
5. feature IPV6 1782 (0.008694)
6. feature FORTIFY_SOURCE 5334 (0.005939)
7. feature GENERIC_ALLOCATOR 11909 (0.005908)
8. feature UBSAN_ALIGNMENT 2727 (0.003383)
9. feature INFINIBAND_ADDR_TRANS 6303 (0.003307)
10. feature UBSAN_NULL 2734 (0.001326)
11. feature UEVENT_HELPER_PATH 1356 (0.001171)
12. feature TTY_PRINTK 689 (0.000877)
13. feature CRC32_SLICEBY8 8756 (0.000728)
14. feature PARIDE_EPIA 3011 (0.000655)
15. feature BATTERY_OLPC 11041 (0.000655)
16. feature FPGA_MGR_SOCFPGA 2017 (0.000634)
17. feature PKCS7_MESSAGE_PARSER 10230 (0.000631)
18. feature DRM_TTM 3278 (0.000609)
19. feature MCP4725 10516 (0.000607)
20. feature DRM_DEBUG_MM_SELFTEST 2697 (0.000595)
Accuracy score: 1.00
Precision score: 1.00
Recall score: 1.00
F1 score: 1.00


In [13]:
compilation_failures = len(combined_pd.query("vmlinux == -1").index)
compilation_successes = len(combined_pd.query("vmlinux != -1").index)
n_compilations = len(combined_pd.index)
assert(compilation_successes + compilation_failures == n_compilations)
compilation_failure_percentage = (compilation_failures / n_compilations) * 100
print("compilation failures:", compilation_failures, "out of", n_compilations, "(", compilation_failure_percentage, "%)")

compilation failures: 3534 out of 69459 ( 5.087893577506155 %)
